### Receipt readout
This notebook is used to extract the abbreviated product names and prices from the receipts and writes the output into a dataframe.

In [132]:
# import libraries
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np
from os import listdir
from os.path import join

SA_KEY=os.getenv("GOOGLE_SA_KEY")
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = SA_KEY

In [133]:
# Googles OCR function
def detect_text(path):
    """Detects text in the file."""
    from google.cloud import vision

    client = vision.ImageAnnotatorClient()

    with open(path, "rb") as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations
    
    '''
    # commented out to supress printed output of the function
    print("Texts:")
    for text in texts:
        print(f'\n"{text.description}"')

        vertices = [
            f"({vertex.x},{vertex.y})" for vertex in text.bounding_poly.vertices
        ]

        print("bounds: {}".format(",".join(vertices)))
    '''
    if response.error.message:
        raise Exception(
            "{}\nFor more info on error messages, check: "
            "https://cloud.google.com/apis/design/errors".format(response.error.message)
            )
    return response

In [134]:
# find all receipts in the specified path and sort them in ascending order
path = 'rewe_scanned/'
files = listdir(path)
files.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
files

['Rewe_1.jpg',
 'Rewe_2.jpg',
 'Rewe_3.jpg',
 'Rewe_4.jpg',
 'Rewe_5.jpg',
 'Rewe_6.jpg',
 'Rewe_7.jpg',
 'Rewe_8.jpg',
 'Rewe_9.jpg',
 'Rewe_10.jpg',
 'Rewe_11.jpg',
 'Rewe_12.jpg',
 'Rewe_13.jpg',
 'Rewe_14.jpg',
 'Rewe_15.jpg',
 'Rewe_16.jpg']

In [135]:
def process_receipts(path,filename):
    '''
    This function takes an image as input and creates a dataframe that contains
    information about the product names and the amount of money that was spent
    on the products. 
    '''
    # Apply function to an receipt
    response = detect_text(join(path,filename))

    # The text_annotations contain the recognized text and the corresponding bounding boxes
    # the first entry contains the whole text from the receipt and the consecutive entries
    # contain the text/coordinates from the individual bounding boxes
    texts = response.text_annotations

    # Build dataframe, where bl: bottom_left, br: bottom_right, tr: top_right, tl: top_left
    # denote the corners of the BBs
    columns = ["String", "x_bl", "y_bl", "x_br", "y_br","x_tr","y_tr","x_tl","y_tl"] # uncomment if you need x coords as well
    #columns = ["String", "y_bl", "y_br","y_tr","y_tl"]
    df = pd.DataFrame(columns=columns)

    for i, text in enumerate(texts):
        df.loc[i, "String"] = text.description
        for j in range(4):
            df.iloc[i,2*j+1] = text.bounding_poly.vertices[j].x  # uncomment if you need x coords as well 
            #df.iloc[i,j+1] = text.bounding_poly.vertices[j].y
            df.iloc[i,2*j+2] = text.bounding_poly.vertices[j].y  # uncomment if you need x coords as well

    # convert the coords to integers for calculation of the mean BB positions
    df[['y_bl','y_br','y_tr','y_tl']] = df[['y_bl','y_br','y_tr','y_tl']].astype('int')
    # calulate mean BB positions
    df['mean_y'] = df.eval('(y_bl+y_br+y_tr+y_tl)/4')

    # sort DF by mean height to match text that appears in the same line
    df = df.sort_values(by=['mean_y']).reset_index(drop=True)

    # convert the coords to integers for calculation of the mean BB positions
    df[['y_bl','y_br','y_tr','y_tl']] = df[['y_bl','y_br','y_tr','y_tl']].astype('int')
    # calulate mean BB positions
    df['mean_y'] = df.eval('(y_bl+y_br+y_tr+y_tl)/4')

    # sort DF by mean height to match text that appears in the same line
    df = df.sort_values(by=['mean_y']).reset_index(drop=True)

    # select only the block of the receipt where the products are listed
    product_list_start_ind = int(df[df.String== 'EUR'].index.values[0])+1
    product_list_end_ind = int(df[df.String=='SUMME'].index.values)
    df_products = df[product_list_start_ind:product_list_end_ind]

    # Create empty list and dataframe to fill it later on
    shown_indices = []
    columns = ['product_name','price']
    df_cleaned = pd.DataFrame(columns=columns)
    counter = 1 # used to build the cleaned data frame row by row

    # check if consecutive rows in df_products belong to the same line in the receipt
    for i in df_products['mean_y']:
        condition = (df_products['mean_y'] >= i) & (df_products['mean_y'] < i + 10)
        indices = df_products.index[condition] # store the indices that fulfill the condition
        
        # check if the indices have been shown before
        if not any(idx in shown_indices for idx in indices):
            # only keep lines that end with A,B or * and write product_names and prices into df_cleaned
            chars = ['A','B','*']
            selected = df_products.loc[indices].sort_values(by=['x_bl'])['String']
            if selected.iloc[-1] in chars:
                if selected.iloc[-1] == '*':
                    df_cleaned.loc[counter,'product_name'] = ' '.join(selected.iloc[:-3])
                    df_cleaned.loc[counter,'price'] = selected.iloc[-3].replace(',','.')
                else:
                    df_cleaned.loc[counter,'product_name'] = ' '.join(selected.iloc[:-2])
                    df_cleaned.loc[counter,'price'] = selected.iloc[-2].replace(',','.')
                counter += 1
            shown_indices.extend(indices)

    # convert the price column to floats
    df_cleaned.price = df_cleaned.price.astype('float')
    df_cleaned['receipt_id'] = filename

    return df_cleaned

In [136]:
df_list = []

for file in files:
    df = process_receipts(path,file)
    df_list.append(df)

df_all = pd.concat(df_list,ignore_index=True)


In [137]:
df_all

,product_name,price,receipt_id
0,VOLLKORNTORTILLA,2.58,Rewe_1.jpg
1,TOFU NATUR,4.38,Rewe_1.jpg
2,HAUCHSCHN CURRY,3.18,Rewe_1.jpg
3,HONEYPOMELO PINK,2.99,Rewe_1.jpg
4,DINKEL - CRUNCHY,2.59,Rewe_1.jpg
...,...,...,...
146,MIXERY ICED BLUE,1.29,Rewe_16.jpg
147,"PFAND 0,08 EUR",0.08,Rewe_16.jpg
148,ORANGE SPRITZ,1.49,Rewe_16.jpg
149,"PFAND 0 , 25 EURO 0 ,",25.00,Rewe_16.jpg


In [138]:
df_all.to_csv('scanned_rewe_receipts_1-16.csv')